In [1]:
import random as rndm
import time
import tkinter as tk
from tkinter import messagebox
import math

# Genetics

In [2]:
# Population size
POPULATION = 100

# Number of generations
REPETITION = 100

# Probability of mutation
PM = 0.1

# Probability of crossover
PC = 0.95

In [3]:
def make_gene(size, initial=None):
    if initial is None:
        initial = [0] * size
    mapp = {}
    gene = list(range(1, size+1))
    rndm.shuffle(gene)
    for i in range(size):
        mapp[gene[i]] = i
    for i in range(size):
        if initial[i] != 0 and gene[i] != initial[i]:
            temp = gene[i], gene[mapp[initial[i]]]
            gene[mapp[initial[i]]], gene[i] = temp
            mapp[initial[i]], mapp[temp[0]] = i, mapp[initial[i]]
    return gene

In [4]:
def make_chromosome(size, initial=None):
    if initial is None:
        initial = [[0] * size] * size
    chromosome = []
    for i in range(size):
        chromosome.append(make_gene(size, initial[i]))
    return chromosome


In [5]:
def make_population(count, size, initial=None):
    if initial is None:
        initial = [[0 for _ in range(size)] for _ in range(size)]
    population = []
    for _ in range(count):
        population.append(make_chromosome(size, initial))
    return population


In [6]:
def get_fitness(size, chromosome):
    fitness = 0
    subgrid_height = 2 if size == 6 else int(size**0.5)
    subgrid_width = 3 if size == 6 else int(size**0.5)
    for i in range(len(chromosome)):  
        seen = {}
        for j in range(len(chromosome[i])):  
            if chromosome[j][i] in seen:
                seen[chromosome[j][i]] += 1
            else:
                seen[chromosome[j][i]] = 1
        for key in seen:
            fitness -= (seen[key] - 1)
    for m in range(subgrid_height):
        for n in range(subgrid_width):
            seen = {}
            for i in range(subgrid_width * n, min(subgrid_width * (n + 1), len(chromosome))): 
                for j in range(subgrid_height * m, min(subgrid_height * (m + 1), len(chromosome[i]))): 
                    if chromosome[j][i] in seen:
                        seen[chromosome[j][i]] += 1
                    else:
                        seen[chromosome[j][i]] = 1
            for key in seen:
                fitness -= (seen[key] - 1)
    return fitness



In [7]:
def pch(ch):
    for i in range(9):
        for j in range(9):
            print(ch[i][j], end=" ")
        print("")


In [8]:
def crossover(ch1, ch2):
    new_child_1 = []
    new_child_2 = []
    for i in range(len(ch1)): 
        x = rndm.randint(0, 1)
        if x == 1:
            new_child_1.append(ch1[i])
            new_child_2.append(ch2[i])
        elif x == 0:
            new_child_2.append(ch1[i])
            new_child_1.append(ch2[i])
    return new_child_1, new_child_2


In [9]:
def mutation(size, ch, pm, initial):
    for i in range(size):
        x = rndm.randint(0, 100)
        if x < pm * 100:
            ch[i] = make_gene(size, initial[i])
    return ch


In [10]:
def read_puzzle_from_array(array):
    return array

In [11]:
def r_get_mating_pool(population, size):
    fitness_list = []
    pool = []
    for chromosome in population:
        fitness = get_fitness(size, chromosome)  
        fitness_list.append((fitness, chromosome))
    fitness_list.sort()
    weight = list(range(1, len(fitness_list) + 1))
    for _ in range(len(population)):
        ch = rndm.choices(fitness_list, weight)[0]
        pool.append(ch[1])
    return pool


In [12]:
def get_offsprings(population, size, initial, pm, pc):
    new_pool = []
    i = 0
    while i < len(population):
        ch1 = population[i]
        ch2 = population[(i + 1) % len(population)]
        x = rndm.randint(0, 100)
        if x < pc * 100:
            ch1, ch2 = crossover(ch1, ch2)
        new_pool.append(mutation(size, ch1, pm, initial))
        new_pool.append(mutation(size, ch2, pm, initial))
        i += 2
    return new_pool


In [13]:
def create_grid(parent, size):
    labels = []
    for i in range(size):
        row = []
        for j in range(size):
            label = tk.Label(master=parent, text=" ", borderwidth=1, relief="solid", width=2)
            label.grid(row=i, column=j)
            row.append(label)
        labels.append(row)
    return labels


In [14]:
def update_grid(labels, size, state):
    for i in range(size):
        for j in range(size):
            labels[i][j].config(text=str(state[i][j]))


The main genetics algorithm

In [15]:
def genetic_algorithm(initial_array, num_generations, pm, pc):
    initial = read_puzzle_from_array(initial_array)
    size = len(initial)
    population = make_population(POPULATION, size, initial)
    for generation in range(num_generations):
        mating_pool = r_get_mating_pool(population, size)
        rndm.shuffle(mating_pool)
        population = get_offsprings(mating_pool, size, initial, pm, pc)
        fit = [get_fitness(size, c) for c in population]  # Adjusted to two arguments
        m = max(fit)
        best_solution = population[fit.index(m)]
        yield best_solution  # Yield the best solution found so far
        if m == 0:
            break

# Main

In [16]:
def button1_click():
    root.withdraw()  # Hide the main window
    grid_window(4)  # Open a new window with a 4x4 grid

def button2_click():
    root.withdraw()  # Hide the main window
    grid_window(6)  # Open a new window with a 6x6 grid

def button3_click():
    root.withdraw()  # Hide the main window
    grid_window(9)  # Open a new window with a 6x6 grid

In [17]:
def submit_click():
    print("Submit clicked")
    for i in range(size):
        for j in range(size):
            if entries[i][j].cget("bg") == "red":
                messagebox.showinfo("Validation Error", "The board needs to be validated first.")
                return
    grid = [[entries[i][j].get() for j in range(size)] for i in range(size)]
    for row in grid:
        print(' '.join(row))
    new_window.destroy()  # Close the current window
    display_grid_window(grid)  # Open a new window with the same numbers


In [18]:

def validate_entry(cell, i, j):
    value = cell.get()
    if not value:  # Check if the value is empty
        cell.configure(bg="white")
        return True
    if not value.isdigit() or not 1 <= int(value) <= size:
        cell.configure(bg="red")
        return False
    value = int(value)
    for k in range(size):
        if (k != j and entries[i][k].get() == str(value)) or (k != i and entries[k][j].get() == str(value)):
            cell.configure(bg="red")
            return False
    # Check if the value is unique in its sub-grid
    if size == 4:
        start_row, start_col = i - i % 2, j - j % 2
        for x in range(2):
            for y in range(2):
                if entries[start_row + x][start_col + y].get() == str(value) and (start_row + x != i or start_col + y != j):
                    cell.configure(bg="red")
                    return False
    elif size == 6:
        start_row, start_col = i - i % 2, j - j % 3
        for x in range(2):
            for y in range(3):
                if entries[start_row + x][start_col + y].get() == str(value) and (start_row + x != i or start_col + y != j):
                    cell.configure(bg="red")
                    return False
    elif size == 9:
        start_row, start_col = i - i % 3, j - j % 3
        for x in range(3):
            for y in range(3):
                if entries[start_row + x][start_col + y].get() == str(value) and (start_row + x != i or start_col + y != j):
                    cell.configure(bg="red")
                    return False
    cell.configure(bg="white")
    return True

In [19]:
def on_key_release(cell, i, j):
    def inner(event):
        validate_entry(cell, i, j)
    return inner

## Panel 2 (Board entry)

In [20]:
def grid_window(s):
    global entries, size, new_window
    size = s
    entries = [[None for _ in range(size)] for _ in range(size)]
    new_window = tk.Toplevel(root)
    new_window.geometry("500x500")

    grid_frame = tk.Frame(new_window)  # Create a frame to hold the grid
    grid_frame.pack(side='top', expand=True, fill='both')  # Place the frame at the top of the window and allow it to expand and fill the space

    for i in range(size):
        for j in range(size):
            cell = tk.Entry(grid_frame, borderwidth=1, relief="solid", font=("Arial", 25))
            cell.grid(row=i, column=j, sticky='nsew')
            cell.bind("<KeyRelease>", on_key_release(cell, i, j))
            entries[i][j] = cell
    grid_frame.grid_columnconfigure(list(range(size)), weight=1)
    grid_frame.grid_rowconfigure(list(range(size)), weight=1)

    submit_frame = tk.Frame(new_window)  # Create a frame to hold the submit button
    submit_frame.pack(side='bottom', expand=True, fill='x')  # Place the frame at the bottom of the window and allow it to expand and fill the space

    submit_button = tk.Button(submit_frame, text="Submit", command=submit_click, height=2, width=10, font=("Arial", 10, "bold"))  # Set the height, width, and font of the button
    submit_button.pack(side='right', expand=True)  # Place the button at the right of the frame and allow it to expand


## Backtracking

In [21]:
def is_valid(board, row, col, num):
    grid_size = len(board)
    for x in range(grid_size):
        if board[row][x] == num:
            return False

    for x in range(grid_size):
        if board[x][col] == num:
            return False

    if int(math.sqrt(grid_size))**2 == grid_size:  # If the grid size is a perfect square
        startRow = row - row % int(math.sqrt(grid_size))
        startCol = col - col % int(math.sqrt(grid_size))
        for i in range(int(math.sqrt(grid_size))):
            for j in range(int(math.sqrt(grid_size))):
                if board[i + startRow][j + startCol] == num:
                    return False
    else:  # If the grid size is not a perfect square
        subgrid_height = max(i for i in range(1, int(math.sqrt(grid_size)) + 1) if grid_size % i == 0)
        subgrid_width = grid_size // subgrid_height
        startRow = row - row % subgrid_height
        startCol = col - col % subgrid_width
        for i in range(subgrid_height):
            for j in range(subgrid_width):
                if board[i + startRow][j + startCol] == num:
                    return False
    return True

In [22]:
iteration_count = 0
def solve_sudoku_Backtrack(board):
    global iteration_count
    iteration_count += 1
    size = len(board)
    for i in range(size):
        for j in range(size):
            if board[i][j] == 0:
                for num in range(1, size + 1):
                    if is_valid(board, i, j, num):
                        board[i][j] = num
                        cells[i][j].config(text=str(board[i][j]))  # Update the cell in the GUI
                        root.update_idletasks()  # Update idle tasks
                        root.update()  # Update the GUI
                        time.sleep(0.01)  # Add a delay
                        if solve_sudoku_Backtrack(board):
                            return True
                        board[i][j] = 0
                        cells[i][j].config(text='')  # Update the cell in the GUI
                        root.update_idletasks()  # Update idle tasks
                        root.update()  # Update the GUI
                return False
    return True


## Panel 3 (Sudoku board)

In [23]:
def display_grid_window(grid):
    global cells, original_grid  
    size = len(grid)
    new_window = tk.Toplevel(root)
    new_window.geometry("500x500")

    iteration_label = tk.Label(new_window, text="Iterations: 0")  # Move the iteration_label to the new window
    iteration_label.pack()

    grid_frame = tk.Frame(new_window)
    grid_frame.pack(side='top', expand=True, fill='both')

    cells = [[tk.Label(grid_frame, text=grid[i][j], borderwidth=1, relief="solid", font=("Arial", 25)) for j in range(size)] for i in range(size)]
    original_grid = [[grid[i][j] for j in range(size)] for i in range(size)]  # Store the original grid
    for i in range(size):
        for j in range(size):
            cells[i][j].grid(row=i, column=j, sticky='nsew')
    grid_frame.grid_columnconfigure(list(range(size)), weight=1)
    grid_frame.grid_rowconfigure(list(range(size)), weight=1)

    def solve_Backtraking():
        global iteration_count
        iteration_count = 0  # Reset the iteration count
        board = [[int(cells[i][j].cget("text")) if cells[i][j].cget("text") != '' else 0 for j in range(size)] for i in range(size)]
        if solve_sudoku_Backtrack(board):
            for i in range(size):
                for j in range(size):
                    cells[i][j].config(text=str(board[i][j]))
        else:
            messagebox.showinfo("No Solution", "There is no solution to the puzzle.")
        iteration_label.config(text=f"Iterations: {iteration_count}")  # Update the label text
    def refresh():
        for i in range(size):
            for j in range(size):
                cells[i][j].config(text=str(original_grid[i][j]))  # Reset the cell to its original state
    generation_count = 0  # Add a global variable to keep track of the generation count

    def solve_genetics():
        global generation_count
        generation_count = 0  # Reset the generation count
        board = [[int(cells[i][j].cget("text")) if cells[i][j].cget("text") != '' else 0 for j in range(size)] for i in range(size)]
        for solution in genetic_algorithm(board, num_generations=1000,pm=PM, pc=PC):  
            generation_count += 1
            for i in range(size):
                for j in range(size):
                    cells[i][j].config(text=str(solution[i][j]))
            root.update_idletasks()  # Update idle tasks
            root.update()  # Update the GUI
            time.sleep(0.01)  # Add a delay
        iteration_label.config(text=f"Generations: {generation_count}")  # Update the label text
    button_frame = tk.Frame(new_window)
    button_frame.pack(side='bottom', expand=True, fill='x')

    button1 = tk.Button(button_frame, text="Backtraking", height=2, width=10, font=("Arial", 10, "bold"), command=solve_Backtraking)
    button1.pack(side='left', expand=True)

    button2 = tk.Button(button_frame, text="Genetics", height=2, width=10, font=("Arial", 10, "bold"), command=solve_genetics)
    button2.pack(side='left', expand=True)

    button3 = tk.Button(button_frame, text="Refresh", height=2, width=10, font=("Arial", 10, "bold"), command=refresh)
    button3.pack(side='left', expand=True)

## Panel 1 (Main)

In [24]:
root = tk.Tk()
root.geometry("500x200")  # Adjust the size of the window

label = tk.Label(root, text="Choose grid size", font=("Arial", 20, "bold"))
label.pack()  # Add the label to the window

frame = tk.Frame(root)  # Create a frame to hold the buttons
frame.pack(side='bottom', expand=True)  # Place the frame at the bottom of the window and allow it to expand

button1 = tk.Button(frame, text="4 * 4", command=button1_click, height=2, width=10, font=("Arial", 10, "bold"))  # Set the height, width, and font of the button
button1.pack(side='left', expand=True)  # Place the button at the left of the frame and allow it to expand

button2 = tk.Button(frame, text="6 * 6", command=button2_click, height=2, width=10, font=("Arial", 10, "bold"))  # Set the height, width, and font of the button
button2.pack(side='left', expand=True)  # Place the button at the left of the frame and allow it to expand

button3 = tk.Button(frame, text="9 * 9", command=button3_click, height=2, width=10, font=("Arial", 10, "bold"))  # Set the height, width, and font of the button
button3.pack(side='left', expand=True)  # Place the button at the left of the frame and allow it to expand

root.mainloop()

Submit clicked
        
    4    
 6     3  
        
   1     
        
    9    
        
        
